In [4]:
import mysql.connector

def run_query(query):
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='Techlabs#2025',
            database='techlabs_projekt'
        )

        cursor = connection.cursor(dictionary=True)
        cursor.execute(query)
        results = cursor.fetchall()

        return results

    except mysql.connector.Error as err:
        print(f"Fehler: {err}")
        return None

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

            


In [5]:
import pandas as pd
import mysql.connector

# CSV laden
df = pd.read_csv("C:/Users/49173/Downloads/Wohnberechtigte-Bevölkerung-Export-bis-2023_0 (1).csv", sep=";")

# Spalten aufbereiten
df["jahr"] = pd.to_datetime(df["ZEIT"], dayfirst=True).dt.year
df = df.rename(columns={"RAUM": "stadtteil", "WERT": "bevoelkerung"})

# DB Verbindung
db = mysql.connector.connect(
    host="localhost", user="root", password="Techlabs#2025", database="techlabs_projekt"
)
cursor = db.cursor()

for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO einwohner (jahr, stadtteil, bevoelkerung)
        VALUES (%s, %s, %s)
    """, (int(row["jahr"]), row["stadtteil"], int(row["bevoelkerung"])))

db.commit()
cursor.close()
db.close()


In [6]:
if __name__ == "__main__":
    query = "SELECT * FROM adressen LIMIT 5;"
    results = run_query(query)

    if results:
        for row in results:
            print(row)


{'id': 1, 'einrichtung_id': 1, 'funktion_key': 'SSA', 'funktion': 'Sportstättenadresse', 'vorname': '', 'nachname': '', 'titel': '', 'strasse': 'Bonhoefferstr. 54', 'hausnr': '', 'plz': '48151', 'ort': 'Münster'}
{'id': 2, 'einrichtung_id': 2, 'funktion_key': 'SSA', 'funktion': 'Sportstättenadresse', 'vorname': '', 'nachname': '', 'titel': '', 'strasse': 'Lotharingerstr. 17', 'hausnr': '', 'plz': '48147', 'ort': 'Münster'}
{'id': 3, 'einrichtung_id': 3, 'funktion_key': 'SSA', 'funktion': 'Sportstättenadresse', 'vorname': '', 'nachname': '', 'titel': '', 'strasse': 'Kaiser-Wilh.-Ring 1', 'hausnr': '', 'plz': '48145', 'ort': 'Münster'}
{'id': 4, 'einrichtung_id': 4, 'funktion_key': 'SSA', 'funktion': 'Sportstättenadresse', 'vorname': '', 'nachname': '', 'titel': '', 'strasse': 'Breite Gasse 2-3', 'hausnr': '', 'plz': '48143', 'ort': 'Münster'}
{'id': 5, 'einrichtung_id': 5, 'funktion_key': 'SSA', 'funktion': 'Sportstättenadresse', 'vorname': '', 'nachname': '', 'titel': '', 'strasse': 'M

In [11]:
import geopandas as gpd
import mysql.connector
from shapely import wkt

# 📂 Shapefile laden
gdf = gpd.read_file("C:/Users/49173/Downloads/05515000_shp_stadtteil_statistischer-bezirk/stadtteil_(statistischer_bezirk).shp")

# 🟢 Geometrie in WKT umwandeln
gdf["geometry"] = gdf["geometry"].apply(lambda geom: geom.wkt)

# 🔗 Verbindung zur DB
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Techlabs#2025",
    database="techlabs_projekt"
)
cursor = db.cursor()

# 🟢 Insert-Query
insert_query = """
INSERT INTO stadtteile2 (nr_statist, name, stadtbezirk, shape_area, shape_len, geom_wkt)
VALUES (%s, %s, %s, %s, %s, %s)
"""

# 🟢 Daten einfügen
for _, row in gdf.iterrows():
    cursor.execute(insert_query, (
        row["NR_STATIST"],         # Nummer
        row["NAME_STATI"],         # Name des Bezirks
        row["STADTBEZIR"],         # Stadtbezirk
        float(row["SHAPE_AREA"]),  # Fläche
        float(row["SHAPE_LEN"]),   # Länge
        row["geometry"]            # Geometrie als WKT
    ))

db.commit()
cursor.close()
db.close()
print("✅ Stadtteile importiert!")


C:\Users\49173\AppData\Local\Temp\ipykernel_18452\2355241099.py:9: UserWarning: Geometry column does not contain geometry.
  gdf["geometry"] = gdf["geometry"].apply(lambda geom: geom.wkt)


✅ Stadtteile importiert!


In [13]:
import mysql.connector
import geopandas as gpd
from shapely.geometry import Point
import shapely.wkt

# 🔗 DB Verbindung
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Techlabs#2025",
    database="techlabs_projekt"
)
cursor = db.cursor(dictionary=True)

# 🟢 Stadtteile laden
cursor.execute("SELECT id, name, geom_wkt FROM stadtteile2")
stadtteile = cursor.fetchall()

# 🟢 Einrichtungen mit Geodaten laden
cursor.execute("""
    SELECT e.id, e.name, g.breitengrad, g.laengengrad
    FROM einrichtungen e
    JOIN adressen a ON a.einrichtung_id = e.id
    JOIN geodaten g ON g.adressen_id = a.id
""")
einrichtungen = cursor.fetchall()

cursor.close()
db.close()

print(f"✅ Geladene Stadtteile: {len(stadtteile)}")
print(f"✅ Geladene Einrichtungen: {len(einrichtungen)}")

# 🧪 CRS Debug: Teste erste Polygon-Koordinaten
first_poly = shapely.wkt.loads(stadtteile[0]["geom_wkt"])
print("🔍 Erste Polygon Bounds:", first_poly.bounds)

# 🧪 Debug-Funktion
def finde_stadtteil(lon, lat):
    punkt = Point(lon, lat)
    for stadtteil in stadtteile:
        poly = shapely.wkt.loads(stadtteil["geom_wkt"])
        if poly.contains(punkt):
            return stadtteil["name"]
    return None

# 🔍 Test mit ersten 5 Einrichtungen
print("\n🔬 Teste die ersten 5 Einrichtungen:")
for einr in einrichtungen[:5]:
    # Variante A: (lon, lat)
    stadtteil_a = finde_stadtteil(einr["laengengrad"], einr["breitengrad"])
    # Variante B: (lat, lon) -> falls vertauscht
    stadtteil_b = finde_stadtteil(einr["breitengrad"], einr["laengengrad"])

    print(f"🏢 {einr['name']} ({einr['id']})")
    print(f"  Koordinaten: Lng={einr['laengengrad']} Lat={einr['breitengrad']}")
    print(f"  Stadtteil (Lng/Lat): {stadtteil_a}")
    print(f"  Stadtteil (Lat/Lng): {stadtteil_b}")
    print("-----")

# 🔥 Zähle, wie viele zugewiesen werden konnten
zugewiesen = 0
nicht_gefunden = 0

for einr in einrichtungen:
    if finde_stadtteil(einr["laengengrad"], einr["breitengrad"]):
        zugewiesen += 1
    else:
        nicht_gefunden += 1

print(f"\n📊 Statistik:")
print(f"✅ Stadtteil gefunden für {zugewiesen} Einrichtungen")
print(f"❌ Kein Stadtteil gefunden für {nicht_gefunden} Einrichtungen")


✅ Geladene Stadtteile: 45
✅ Geladene Einrichtungen: 95
🔍 Erste Polygon Bounds: (405614.8729999997, 5756678.196, 406459.01400000043, 5757698.635)

🔬 Teste die ersten 5 Einrichtungen:
🏢 Aaseesporthalle (105) (1)
  Koordinaten: Lng=7.6032241 Lat=51.9412819
  Stadtteil (Lng/Lat): None
  Stadtteil (Lat/Lng): None
-----
🏢 Adolph-Kolping-Schule (Sporthalle)(402) (2)
  Koordinaten: Lng=7.6320056 Lat=51.9659451
  Stadtteil (Lng/Lat): None
  Stadtteil (Lat/Lng): None
-----
🏢 Aegidii-Ludgeri-Schule (101) (4)
  Koordinaten: Lng=7.6237296 Lat=51.9590769
  Stadtteil (Lng/Lat): None
  Stadtteil (Lat/Lng): None
-----
🏢 Anne-Frank-Schule (404) (5)
  Koordinaten: Lng=7.6537947 Lat=51.9564321
  Stadtteil (Lng/Lat): None
  Stadtteil (Lat/Lng): None
-----
🏢 Annette-v.-Dro.-Hülsh.-Sch. GR (301) (6)
  Koordinaten: Lng=7.6807536 Lat=51.9213159
  Stadtteil (Lng/Lat): None
  Stadtteil (Lat/Lng): None
-----

📊 Statistik:
✅ Stadtteil gefunden für 0 Einrichtungen
❌ Kein Stadtteil gefunden für 95 Einrichtungen


In [14]:
import geopandas as gpd
import mysql.connector

# 📂 Shapefile laden
gdf = gpd.read_file("C:/Users/49173/Downloads/05515000_shp_stadtteil_statistischer-bezirk/stadtteil_(statistischer_bezirk).shp")

# 🔍 Aktuelles CRS prüfen
print("Aktuelles CRS:", gdf.crs)

# ✅ Umwandeln in WGS84, falls nötig
if gdf.crs is None:
    # Falls kein CRS gesetzt ist, müssen wir das Original-CRS kennen
    # Münster Shapefiles nutzen meistens EPSG:25832 (ETRS89 / UTM Zone 32N)
    gdf.set_crs(epsg=25832, inplace=True)
if gdf.crs.to_epsg() != 4326:
    gdf = gdf.to_crs(epsg=4326)

print("Neues CRS:", gdf.crs)
print("Bounds nach Umwandlung:", gdf.total_bounds)

# Geometrie in WKT umwandeln
gdf["geometry"] = gdf["geometry"].apply(lambda geom: geom.wkt)

# 🔗 DB-Verbindung
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Techlabs#2025",
    database="techlabs_projekt"
)
cursor = db.cursor()

# Alte Stadtteile löschen (optional)
cursor.execute("TRUNCATE TABLE stadtteile2")

# Neue Daten einfügen
insert_query = """
INSERT INTO stadtteile2 (nr_statist, name, stadtbezirk, shape_area, shape_len, geom_wkt)
VALUES (%s, %s, %s, %s, %s, %s)
"""

for _, row in gdf.iterrows():
    cursor.execute(insert_query, (
        row["NR_STATIST"],
        row["NAME_STATI"],
        row["STADTBEZIR"],
        float(row["SHAPE_AREA"]),
        float(row["SHAPE_LEN"]),
        row["geometry"]
    ))

db.commit()
cursor.close()
db.close()

print("✅ Stadtteile neu importiert mit EPSG:4326")


Aktuelles CRS: EPSG:25832
Neues CRS: EPSG:4326
Bounds nach Umwandlung: [ 7.47378495 51.84014408  7.77436596 52.06002443]


C:\Users\49173\AppData\Local\Temp\ipykernel_18452\2632180965.py:22: UserWarning: Geometry column does not contain geometry.
  gdf["geometry"] = gdf["geometry"].apply(lambda geom: geom.wkt)


✅ Stadtteile neu importiert mit EPSG:4326


In [15]:
import mysql.connector
from shapely import wkt
from shapely.geometry import Point

# DB-Verbindung
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Techlabs#2025",
    database="techlabs_projekt"
)
cursor = db.cursor(dictionary=True)

# Stadtteile laden
cursor.execute("SELECT id, name, geom_wkt FROM stadtteile2")
stadtteile = cursor.fetchall()
for st in stadtteile:
    st["geom"] = wkt.loads(st["geom_wkt"])

# Einrichtungen mit Koordinaten laden
cursor.execute("""
SELECT e.id, e.name, g.breitengrad, g.laengengrad
FROM einrichtungen e
JOIN adressen a ON a.einrichtung_id = e.id
JOIN geodaten g ON g.adressen_id = a.id
""")
einrichtungen = cursor.fetchall()

print(f"✅ Geladene Stadtteile: {len(stadtteile)}")
print(f"✅ Geladene Einrichtungen: {len(einrichtungen)}")

# Test für 5 Einrichtungen
for einr in einrichtungen[:5]:
    point = Point(einr["laengengrad"], einr["breitengrad"])  # (lng, lat)
    matched = None
    for st in stadtteile:
        if st["geom"].contains(point):
            matched = st["name"]
            break
    print(f"🏢 {einr['name']} → Stadtteil: {matched}")


✅ Geladene Stadtteile: 45
✅ Geladene Einrichtungen: 95
🏢 Aaseesporthalle (105) → Stadtteil: Aaseestadt
🏢 Adolph-Kolping-Schule (Sporthalle)(402) → Stadtteil: Martini
🏢 Aegidii-Ludgeri-Schule (101) → Stadtteil: Aegidii
🏢 Anne-Frank-Schule (404) → Stadtteil: Herz-Jesu
🏢 Annette-v.-Dro.-Hülsh.-Sch. GR (301) → Stadtteil: Angelmodde
